<a href="https://colab.research.google.com/github/fahimku2020/fahimku2020/blob/main/RAG_FAST_MODEL_WITh_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install faiss-cpu
!pip install wikipedia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 30.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=bdb63e176d26677d0c64d424a7d169388dd339cb71098a4ed9c5a20a7ca522b1
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [26]:
import wikipedia
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import nltk
#nltk.download('punkt')
#nltk.download('punkt_tab')
#nltk.download('stopwords')

# 1. Data Loading (Wikipedia) and Preprocessing

def fetch_wikipedia_data(query, num_pages=3):
  pages = wikipedia.search(query, results=num_pages)
  data = ""
  for page in pages:
    content = wikipedia.page(page).content
    return content

def preprocess_text(text):
  stop_words = set(stopwords.words('english'))
  preprocessed_sentences = []
  sentences = sent_tokenize(text)
  for sentence in sentences :
    words = [word.lower() for word in nltk.word_tokenize(sentence) if word.isalnum() and word.lower() not in stop_words]
    preprocessed_sentences.append(" ".join(words))
  return preprocessed_sentences


query = "Amitabh  Bachchan films "
wikipedia_data = fetch_wikipedia_data(query)
#print(wikipedia_data)
sentences = preprocess_text(wikipedia_data)
print(sentences)



['amitabh bachchan indian actor playback singer film producer television host former politician primarily works hindi films', 'made acting debut 1969 saat hindustani narrated mrinal sen bhuvan shome 1969', 'later appeared bhaskar banerjee hrishikesh mukherjee anand 1971 filmfare award best supporting actor', '1973 bachchan played breakthrough role inspector vijay khanna prakash mehra action film zanjeer', 'since appeared several films character name vijay', 'year appeared abhimaan namak haraam', 'latter received filmfare award best supporting actor', 'starred along shashi kapoor yash chopra deewaar earned widespread critical acclaim popularity made filmfare award best actor nomination', 'cited angry young man roles deewaar zanjeer', 'later starred ramesh sippy sholay 1975 considered one greatest indian films time', 'appearing romantic drama kabhie kabhie 1976 bachchan starred manmohan desai highest grosser amar akbar anthony 1977', 'filmfare award best actor performance latter', 'playe

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

def search_and_rerank(query, index, vectorizer, sentences, labels, kmeans, topk=5):
    # 1. Initial Search (TF-IDF & Cosine Similarity)
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, index).flatten()
    top_indices = np.argsort(similarities)[-topk:]  # Get indices of top-k similar sentences
    initial_results = [(sentences[i], similarities[i]) for i in top_indices]


    # 2. Reranking with K-means Clustering
    query_cluster = kmeans.predict(query_vec)[0] # Predict the cluster of the query

    reranked_results = []
    for sentence, similarity in initial_results:
        sentence_index = sentences.index(sentence)  # Need to get the original index
        sentence_cluster = labels[sentence_index]

        if sentence_cluster == query_cluster:  # Boost score if in same cluster
            reranked_results.append((sentence, similarity + 0.2)) # Example boost value, tune as needed
        else:
            reranked_results.append((sentence, similarity)) # Use the same similarity when not in the same cluster

    # 3. Sort Reranked Results
    reranked_results.sort(key=lambda x: x[1], reverse=True)  # Sort by reranked similarity



    return reranked_results[:topk]






# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
index = vectorizer.fit_transform(sentences)

# Mock k-means clustering (replace with actual training)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(index)  # Fit to your sentence vectors
labels = kmeans.labels_

results = search_and_rerank(query, index, vectorizer, sentences, labels, kmeans, topk=15)

for sentence, score in results:
    print(f"Sentence: {sentence}, Score: {score}")

Sentence: films acting credits production credits television documentaries music videos footnotes see also awards nominations received amitabh bachchan references external links amitabh bachchan imdb amitabh bachchan rotten tomatoes, Score: 0.7089477180409769
Sentence: amitabh bachchan also known shahenshah big b bollywood, Score: 0.5444854755760884
Sentence: amitabh bachchan indian actor playback singer film producer television host former politician primarily works hindi films, Score: 0.5356002922471833
Sentence: 1996 started film production company amitabh bachchan corporation whose first release tere mere sapne 1996 hit, Score: 0.39403010091481605
Sentence: bachchan also narrated several films including shatranj ke khilari 1977 lagaan 2001 parineeta 2005 jodhaa akbar 2008 2011 krrish 3 2013, Score: 0.3539777366823486
Sentence: also performed playback singer films like laawaris silsila baghban, Score: 0.33738188762048205
Sentence: 2000 bachchan appeared widely acclaimed supporting r